# build a new custom environmet for reinforcemnt learning


this code re-bilut https://www.dideo.tv/v/yt/bD6V3rcr_54/%D8%AA%D9%85%D8%A7%D8%B4%D8%A7%DB%8C-%D9%88%DB%8C%D8%AF%D8%A6%D9%88-%D8%A7%D8%B2-%D8%AF%DB%8C%D8%AF%D8%A6%D9%88-watch-video-from-dideo


in this custom environment we want to build an agent in which it can adjust the water in 60 seconds, in this game we have 3 actions, turn down, turn up, leave. optimal solution is when water degree become between 39 to 36 

so in order to create a custom environment we have to implement 


the __init__() method for a OpenAI Environment
the step() method for an OpenAI Environment
the reset() method for an OpenAI Environmen

In [ ]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2

In [ ]:
from gym import Env
#creating actions and states by using gym.space
from gym.spaces import Discrete, Box
import numpy as np
import random

class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)   # it return 0,1,2
        # Temperature array and used to show the range of degree of water and Box return us a continous value
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp and shows current degree
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    #this funcion should return state, reward, done and info
    def step(self, action): # action can be oe of 0,1,2 
        # Apply action
        # 0 -1 = -1 reduce 1 degree if action = 0
        # 1 -1 = 0  degree of water will be remain stable
        # 2 -1 = 1  degree will be increased if action is 2
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info
    def render(self):
        # Implement visual evirnment by using PyGame
    def reset(self):
        self.state = 38 + random.randint(-3,3)
        # Reset shower time
        self.shower_length = 60 
        return self.state

    
    
env = ShowerEnv()
#this create a random action among 0,1,2
env.action_space.sample()
#this returns a countinous value
env.observation_space.sample()



In [ ]:
import gym
import random

episodes = 10
for episode in range(1 , episodes+1):
    state = env.reset()
    done = False
    score = 0 
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done , info = env.step(action)
        score+= reward
    print("Episode:{}   score{}".format(episode , score) )

In [ ]:
states = env.observation_space.shape
actions = env.action_space.n

In [ ]:
import numpy as np
from tensorflow.keras.layers import Dense, Falatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential

def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model


model = buid_model(states , actions)
model.summary()

In [ ]:
from rl.agents import DQNAgent
# we have 2 kind of problems: valuesbasedrl and policybasedRl. int this program we solve policybased
from rl.policy import BoltzmanQPolicy
from rl.memory import SequentialMemory


def build_agent(model , actions):
    policy = BoltzmanQPolicy()
    memory = SequentialMemory(limit=50000 , window_length = 1)
    dqn = DQNAgent(model = model , memory = memory , policy = policy,
                  nb_actions = actions , nb_steps_warmup = 10 , targer_model_update =1e-2 )
    return dqn

dqn = build_agent(model , actions)
dqn.compile(Adam(lr = 1e-3) , metrics = ['mse'])
dqn.fit(env , nb_steps = 50000 , visualize = False , verbose = 1 )